In [1]:
import os
import sys
from pathlib import Path

if "workding_dir" not in locals():
    workding_dir = str(Path.cwd().parent)
os.chdir(workding_dir)
sys.path.append(workding_dir)
print("working dir:", workding_dir)

working dir: c:\Users\admin\code\CrediNews


In [2]:
from dotenv import find_dotenv, load_dotenv

found_dotenv = find_dotenv(".env")

if len(found_dotenv) == 0:
    found_dotenv = find_dotenv(".env.example")
print(f"loading env vars from: {found_dotenv}")
load_dotenv(found_dotenv, override=True)

loading env vars from: c:\Users\admin\code\CrediNews\.env


True

In [3]:
# !pip install wandb
import os
os.getenv("WANDB_API_KEY", "empty")[-5:]

'f541b'

In [4]:
import os
from transformers import DataCollatorWithPadding, AutoTokenizer, AutoModelForSequenceClassification

# Change these as needed.
model_name = "FacebookAI/roberta-base"

# Triggers tokenizer download to cache
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Downloading and caching pre-trained model")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Triggers model download to cache
AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

## Training

With everything now downloaded and cached, we can now set up our training function. Our training function defines the training execution for a single hyperparameter configuration. For now we pull these hyperparameters from a config argument, but we’ll see later how this is passed in.

First we get our datasets- we only use the first half of the dev dataset for validation, and leave the rest of testing:

In [5]:
from datasets import load_dataset, concatenate_datasets, Dataset

datasets = load_dataset(
    "csv",
    data_files={
        "train": [
            "dataset/train_data_1.csv",
            "dataset/train_data_2.csv",
            "dataset/train_data_3.csv",
            "dataset/train_data_4.csv",
        ],
        "test": "dataset/test_data.csv",
        "rewritten_train": [
            "dataset/rewritten_train_data_1.csv",
            "dataset/rewritten_train_data_2.csv",
            "dataset/rewritten_train_data_3.csv",
            "dataset/rewritten_train_data_4.csv",
        ],
        "rewritten_test": "dataset/rewritten_test_data.csv",
    },
)

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["processed_full_content"], padding="max_length", truncation=True)

tokenized_datasets = datasets.map(tokenize_function, batched=True)

def get_datasets(config="original"):
    if config == "rewritten":
        train_dataset = tokenized_datasets["rewritten_train"]
        eval_dataset = tokenized_datasets["rewritten_test"]
    elif config == "original":
        train_dataset = tokenized_datasets["train"]
        eval_dataset = tokenized_datasets["test"]
    else:
        train_dataset = concatenate_datasets(
            [tokenized_datasets["train"], tokenized_datasets["rewritten_train"]]
        )
        eval_dataset = concatenate_datasets(
            [tokenized_datasets["test"], tokenized_datasets["rewritten_test"]]
        )

    return train_dataset, eval_dataset

Map:   0%|          | 0/54441 [00:00<?, ? examples/s]

In [6]:
get_datasets(config="original"), get_datasets(config="rewritten"), get_datasets(config="all")

((Dataset({
      features: ['label', 'full_content', 'processed_full_content', 'input_ids', 'attention_mask'],
      num_rows: 54441
  }),
  Dataset({
      features: ['label', 'full_content', 'processed_full_content', 'input_ids', 'attention_mask'],
      num_rows: 6050
  })),
 (Dataset({
      features: ['label', 'full_content', 'processed_full_content', 'input_ids', 'attention_mask'],
      num_rows: 54441
  }),
  Dataset({
      features: ['label', 'full_content', 'processed_full_content', 'input_ids', 'attention_mask'],
      num_rows: 6050
  })),
 (Dataset({
      features: ['label', 'full_content', 'processed_full_content', 'input_ids', 'attention_mask'],
      num_rows: 108882
  }),
  Dataset({
      features: ['label', 'full_content', 'processed_full_content', 'input_ids', 'attention_mask'],
      num_rows: 12100
  })))

In [7]:
def wandb_hp_space(trial):
    return {
        "method": "random",
        "metric": {"name": "accuracy", "goal": "maximize"},
        "parameters": {
            "learning_rate": {"distribution": "uniform", "min": 1e-6, "max": 1e-4},
            "per_device_train_batch_size": {"values": [16, 32]},
        },
    }

In [8]:
from transformers import AutoModelForSequenceClassification
import numpy as np
import evaluate
from transformers import EvalPrediction

# Load the accuracy metric from the evaluate library
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred: EvalPrediction) -> dict:
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    return accuracy

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


In [9]:
from transformers import Trainer, TrainingArguments

def do_grid_search(config):
    train_dataset, eval_dataset = get_datasets(config)
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=4,
        weight_decay=0.01,
    )

    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    return trainer.hyperparameter_search(
        direction="maximize",
        backend="wandb", # or "ray", "optuna", "sigopt"
        hp_space=wandb_hp_space, #ray_hp_space, 
        n_trials=5,
    )

In [10]:
%%time

best_trial = do_grid_search("original")
best_trial

c:\Users\admin\.conda\envs\fake-news\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\admin\AppData\Local\Temp\ipykernel_20608\1062649897.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: y4uiki80
Sweep URL: https://wandb.ai/inflaton-ai/uncategorized/sweeps/y4uiki80


wandb: Agent Starting Run: nzq3a7jq with config:
wandb: 	learning_rate: 1.0171689812737018e-05
wandb: 	per_device_train_batch_size: 16
wandb: Currently logged in as: inflaton-sg (inflaton-ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).


...

wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.693800,0.693522,0.437521
2,0.686800,0.686910,0.562479
3,0.684400,0.685599,0.562479
4,0.684700,0.685331,0.562479


eval/accuracy,▁███
eval/loss,█▂▁▁
eval/runtime,█▇▁▁
eval/samples_per_second,▁▂██
eval/steps_per_second,▁▂██
train/epoch,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇████
train/global_step,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇████
train/grad_norm,▂▃▃▃▂▄▂▃█▃▂▂▅▃▃▃▂▁▃▂▁▂▃▂▁▃▃
train/learning_rate,██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▁▁
train/loss,▁███████▇█▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇
eval/accuracy,0.56248


CPU times: total: 8h 30min 42s
Wall time: 9h 27min 1s


BestRun(run_id='x5xvwstt', objective=0.9937190082644628, hyperparameters={'learning_rate': 1.9554285493551877e-05, 'per_device_train_batch_size': 16, 'assignments': {}, 'metric': 'eval/loss'}, run_summary=None)

In [11]:
%%time

best_trial_combined = do_grid_search("combined")
best_trial_combined

c:\Users\admin\.conda\envs\fake-news\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\admin\AppData\Local\Temp\ipykernel_20608\1062649897.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create sweep with ID: 86aapprs
Sweep URL: https://wandb.ai/inflaton-ai/uncategorized/sweeps/86aapprs


wandb: Agent Starting Run: nr9jb96s with config:
wandb: 	learning_rate: 7.690367865493667e-05
wandb: 	per_device_train_batch_size: 16


Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).


...

wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.205200,0.183309,0.949587
2,0.128400,0.151630,0.953719
3,0.084300,0.116110,0.963306
4,0.060400,0.107093,0.966529


eval/accuracy,▁▃▇█
eval/loss,█▅▂▁
eval/runtime,▇█▁▂
eval/samples_per_second,▂▁█▇
eval/steps_per_second,▂▁█▇
train/epoch,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇████
train/global_step,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇████
train/grad_norm,▂▂▂▁▁▁▁▂▁▁▁▂█▃▇▁▁▂▁▁▁▁▁▂▃▄▁
train/learning_rate,██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▁▁
train/loss,█▆▅▆▆▆▅▅▄▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
eval/accuracy,0.96653


CPU times: total: 21h 30min 4s
Wall time: 23h 49min 45s


BestRun(run_id='nxu2t2rk', objective=0.9665289256198347, hyperparameters={'learning_rate': 4.489344564153318e-05, 'per_device_train_batch_size': 32, 'assignments': {}, 'metric': 'eval/loss'}, run_summary=None)

In [12]:
%%time

best_trial_rewritten = do_grid_search("rewritten")
best_trial_rewritten

c:\Users\admin\.conda\envs\fake-news\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\admin\AppData\Local\Temp\ipykernel_20608\1062649897.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create sweep with ID: 5dlzufk5
Sweep URL: https://wandb.ai/inflaton-ai/uncategorized/sweeps/5dlzufk5


wandb: Agent Starting Run: e41ulyzv with config:
wandb: 	learning_rate: 6.500576306112583e-05
wandb: 	per_device_train_batch_size: 32


Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).


...

wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.202200,0.165347,0.933388
2,0.164400,0.160493,0.942314
3,0.126300,0.236850,0.939835
4,0.112800,0.225253,0.947603


eval/accuracy,▁▅▄█
eval/loss,▁▁█▇
eval/runtime,▄▆█▁
eval/samples_per_second,▅▃▁█
eval/steps_per_second,▅▃▁█
train/epoch,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇████
train/global_step,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇████
train/grad_norm,▄▂▂▂▃▂▃▅▂▅▁▆▃▄▇▃▆▂▁▁█▇▁▆▁▁▃
train/learning_rate,██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▁▁
train/loss,█▅▅▄▄▄▃▃▃▃▂▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁
eval/accuracy,0.9476


CPU times: total: 6h 23min 48s
Wall time: 7h 6min 56s


BestRun(run_id='7wcm5o65', objective=0.947603305785124, hyperparameters={'learning_rate': 1.2493646207932298e-05, 'per_device_train_batch_size': 16, 'assignments': {}, 'metric': 'eval/loss'}, run_summary=None)